# ValyuContextRetriever

>[Valyu](https://valyu.ai/) is a platform for context retrieval and search.

We can use this as a [retriever](/docs/how_to#retrievers). This notebook shows functionality specific to this integration.

### Integration details

import {ItemTable} from "@theme/FeatureTables";

<ItemTable category="external_retrievers" item="ValyuContextRetriever" />

## Setup

In [ ]:
%pip install -qU langchain-valyu

We also need to set our Valyu API key.

In [ ]:
import os

valyu_api_key = os.environ["VALYU_API_KEY"]

## Instantiation

Now we can instantiate our retriever:

In [ ]:
from langchain_valyu import ValyuContextRetriever

retriever = ValyuContextRetriever(valyu_api_key=os.environ["VALYU_API_KEY"])

## Usage

In [ ]:
query = "What are the benefits of renewable energy?"
docs = retriever.invoke(query)

for doc in docs:
    print(doc.page_content)
    print(doc.metadata)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)

llm = ChatOpenAI(model="gpt-4o-mini")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)